In [1]:
#needed after restructuring of directory
import os
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
import itertools

In [2]:
# change directory to src so that we can import functions
os.chdir(os.path.join("..","..",".."))
from functions import *

In [3]:
# back to repo root
os.chdir("..")

In [4]:
#importing csv fight data and saving as dataframes
ufc_fights = pd.read_csv('src/models/buildingMLModel/data/processed/ufc_fights.csv',low_memory=False)
ufcfighterscrap =pd.read_csv('src/models/buildingMLModel/data/processed/fighter_stats.csv',sep=',',low_memory=False)

In [5]:
#cleaning the methods column for method prediction (counting split decisions as decisions)
ufc_fights['method'] = clean_method_vect(ufc_fights['method'])
ufc_fights['method'].unique()

array(['SUB', 'DEC', 'KO/TKO', 'bullshit'], dtype=object)

In [6]:
#getting rid of rows with incomplete or useless data
#fights with outcome "Win" or "Loss" (no "Draw")
draw_mask=ufc_fights['result'] != 'D' 
#fights where the method of victory is TKO/SUB/DEC (no bullshit)
method_mask=(ufc_fights['method']=='DEC')|(ufc_fights['method']=='SUB')|(ufc_fights['method']=='KO/TKO')
#fights where age is known
age_mask=(ufc_fights['fighter_age']!='unknown')&(ufc_fights['opponent_age']!='unknown')
#fights where height reach is known
height_mask=(ufc_fights['fighter_height']!='unknown')&(ufc_fights['opponent_height']!='unknown')
reach_mask=(ufc_fights['fighter_reach']!='unknown')&(ufc_fights['opponent_reach']!='unknown')
#fights where number of wins is known
wins_mask=(ufc_fights['fighter_wins'] != 'unknown' )& (ufc_fights['opponent_wins'] != 'unknown')
#fights where both fighters have strike statistics (gets rid of UFC debuts)
strikes_mask=(ufc_fights['fighter_inf_sig_strikes_attempts_avg'] != 0)&(ufc_fights['opponent_inf_sig_strikes_attempts_avg'] != 0)
#includes only the fights satisfying these conditions
ufc_fights=ufc_fights[draw_mask&method_mask&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]

record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#THERE MAY BE A PROBLEM IN AGE HEIGHT REACH TO DO WITH STRING VS FLOAT. MAKE SURE THESE ARE ALL THE CORRECT TYPE
#MAYBE WE ARE LOSING PREDICTABILITY HERE

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

#here is the version of punch stats geared for comparing fighter_inf to opponent_abs
punch_statistics_alt=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg',
                     
                     u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg']

fight_math_stats=['4-fighter_score_diff',
 '9-fighter_score_diff',
 '15-fighter_score_diff',
 '1-fight_math',
 '6-fight_math',]

#adding record differences to ufc_fights
record_statistics_diff = []
half_length=int(len(record_statistics)/2)
for i in range(half_length):
    ufc_fights[record_statistics[i]+'_diff_2']=ufc_fights[record_statistics[i]]-ufc_fights[record_statistics[i+half_length]]
    record_statistics_diff.append(record_statistics[i]+'_diff_2')
    
#lets try and improve the greedy algorithm by considering differences. Lets start by replacing height and reach by their differences
ufc_fights['height_diff']=ufc_fights['fighter_height'].apply(float)-ufc_fights['opponent_height'].apply(float)
ufc_fights['reach_diff']=ufc_fights['fighter_reach'].apply(float)-ufc_fights['opponent_reach'].apply(float)
physical_stats_diff = [ 'fighter_age','opponent_age', 'height_diff', 'reach_diff']

#adding punch differences to ufc_fights
punch_statistics_diff = []
half_length=int(len(punch_statistics)/2)
for i in range(half_length):
    ufc_fights[punch_statistics[i]+'_diff_2']=ufc_fights[punch_statistics[i]]-ufc_fights[punch_statistics[i+half_length]]
    punch_statistics_diff.append(punch_statistics[i]+'_diff_2')
    
possible_stats =record_statistics_diff+physical_stats_diff+punch_statistics_diff+fight_math_stats
 
ufc_fights['fighter_age'] = ufc_fights['fighter_age'].apply(float)
ufc_fights['opponent_age'] = ufc_fights['opponent_age'].apply(float)


In [36]:
yr=ufc_fights['method'].to_numpy()

In [37]:
yr = yr.reshape(-1,1)

In [38]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False).fit(yr)
print(ohe.categories_)

yr_onehot = ohe.transform(yr)
yr_onehot

[array(['DEC', 'KO/TKO', 'SUB'], dtype=object)]


array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [39]:
Xr=ufc_fights[possible_stats]

In [40]:
Xr.shape

(4606, 71)

In [41]:
Xr_normalized=(Xr-Xr.mean())/Xr.std()

In [42]:
Xr_normalized.shape

(4606, 71)

In [43]:
Xr_normalized

,fighter_wins_diff_2,fighter_losses_diff_2,fighter_L5Y_wins_diff_2,fighter_L5Y_losses_diff_2,fighter_L2Y_wins_diff_2,fighter_L2Y_losses_diff_2,fighter_ko_wins_diff_2,fighter_ko_losses_diff_2,fighter_L5Y_ko_wins_diff_2,fighter_L5Y_ko_losses_diff_2,...,fighter_abs_distance_strikes_attempts_avg_diff_2,fighter_abs_clinch_strikes_landed_avg_diff_2,fighter_abs_clinch_strikes_attempts_avg_diff_2,fighter_abs_ground_strikes_landed_avg_diff_2,fighter_abs_ground_strikes_attempts_avg_diff_2,4-fighter_score_diff,9-fighter_score_diff,15-fighter_score_diff,1-fight_math,6-fight_math
0,-0.848764,1.683186,-1.722903,0.511262,-1.258366,0.035509,-0.831088,0.730552,-1.144499,0.895769,...,-0.928222,0.336163,0.193969,-0.557716,-0.605906,0.543358,0.112046,0.307839,0.001512,-0.017532
2,-0.635798,0.350411,-1.032603,0.511262,-2.528590,0.905488,-0.416759,0.730552,-0.575339,0.895769,...,-0.093542,1.418345,1.500149,-0.415707,-0.405313,0.656235,0.370023,0.307839,0.001512,-0.017532
3,-2.126556,-2.315140,-1.032603,-1.941218,0.011858,-0.834469,-2.902738,-1.439433,-1.144499,-1.783395,...,-0.078971,-0.169674,-0.124803,-0.021722,0.016085,-0.133903,-0.081437,-0.063785,0.001512,-0.017532
4,0.003098,0.017217,0.002848,0.020766,0.011858,0.035509,0.411901,0.007224,0.562982,0.002714,...,0.161088,-0.641694,-0.610330,-0.073661,-0.105439,-0.698288,-0.661886,-0.541589,0.001512,-0.017532
5,-1.274694,-1.981947,-1.032603,-2.431714,0.011858,-1.704448,-1.245418,-2.886089,-1.144499,-3.569504,...,1.535967,0.235839,0.493881,-0.615497,-0.802605,-0.359657,-0.081437,0.042393,0.001512,-0.017532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6955,-0.422833,-0.649171,-0.687453,-0.960226,-0.623254,-0.834469,-0.416759,-0.716104,-0.575339,-0.890340,...,-0.007854,0.237530,0.041011,0.242875,0.610817,-0.359657,-0.597391,-0.488500,0.001512,-0.017532
6969,0.216063,0.350411,0.347998,0.511262,0.011858,0.905488,0.411901,0.007224,0.562982,0.002714,...,0.518420,1.029040,1.690586,-0.324654,-0.466387,-0.246780,-0.145931,-0.116875,0.001512,-0.017532
6993,0.429029,0.350411,0.693148,0.511262,0.646970,0.035509,0.826231,0.007224,1.132142,0.002714,...,-0.360314,-1.413716,-1.137006,0.165963,0.121159,0.769112,0.628000,0.520196,0.001512,-0.017532
7038,-0.209868,0.683604,-0.342303,1.001758,-0.623254,1.775466,-0.002429,0.007224,-0.006178,0.002714,...,-0.135283,1.632401,0.931073,0.977295,0.867513,-0.359657,-0.210426,-0.169964,0.001512,-0.017532


In [44]:
import torch

In [45]:
import torch.nn as nn

In [46]:
# convert pandas DataFrame (X) and numpy array (y) into PyTorch tensors
Xr_normalized = torch.tensor(Xr_normalized.values, dtype=torch.float32)
yr_onehot = torch.tensor(yr_onehot, dtype=torch.float32)

In [64]:
# using https://machinelearningmastery.com/building-a-multiclass-classification-model-in-pytorch/
# but trying to get softmax to work
class Multiclass(nn.Module):
    def __init__(self, num_features, num_neurons, num_classes):
        super().__init__()
        self.hidden = nn.Linear(num_features, num_neurons)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_neurons, num_classes)
        self.softmax = nn.LogSoftmax(dim = 1)
        
    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.softmax(self.output(x))
        return x
    
model = Multiclass(Xr_normalized.shape[1], 100, yr_onehot.shape[1])

torch.nn.NLLLoss(weight=None, size_average=None, ignore_index=- 100, reduce=None, reduction='mean')[SOURCE]

The negative log likelihood loss. It is useful to train a classification problem with C classes.

If provided, the optional argument weight should be a 1D Tensor assigning weight to each of the classes. This is particularly useful when you have an unbalanced training set.

In [49]:
import torch.optim as optim

#loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [61]:
Xr_normalized

tensor([[-8.4876e-01,  1.6832e+00, -1.7229e+00,  ...,  3.0784e-01,
          1.5116e-03, -1.7532e-02],
        [-6.3580e-01,  3.5041e-01, -1.0326e+00,  ...,  3.0784e-01,
          1.5116e-03, -1.7532e-02],
        [-2.1266e+00, -2.3151e+00, -1.0326e+00,  ..., -6.3786e-02,
          1.5116e-03, -1.7532e-02],
        ...,
        [ 4.2903e-01,  3.5041e-01,  6.9315e-01,  ...,  5.2020e-01,
          1.5116e-03, -1.7532e-02],
        [-2.0987e-01,  6.8360e-01, -3.4230e-01,  ..., -1.6996e-01,
          1.5116e-03, -1.7532e-02],
        [-2.0987e-01,  1.7217e-02, -3.4230e-01,  ..., -3.8232e-01,
          1.5116e-03, -1.7532e-02]])

In [62]:
model(Xr_normalized)

tensor([[-1.2334, -0.8905, -1.2099],
        [-1.5533, -0.8038, -1.0764],
        [-1.4759, -0.7491, -1.2085],
        ...,
        [-1.4749, -0.5784, -1.5586],
        [-1.1115, -0.9779, -1.2213],
        [-1.6214, -0.4949, -1.6463]], grad_fn=<LogSoftmaxBackward0>)

In [50]:
yr_labels = torch.argmax(yr_onehot, dim=1)
yr_labels

tensor([2, 1, 2,  ..., 0, 2, 1])

In [63]:
# this is whats causing the problem... the loss function is not working
loss_fn(model(Xr_normalized),yr_labels)

tensor(nan, grad_fn=<NllLossBackward0>)

In [52]:
# size of input (N x C) is = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# every element in target should have 0 <= value < C
target = torch.tensor([1, 0, 4])

m = nn.LogSoftmax(dim=1)
nll_loss = nn.NLLLoss()
output = nll_loss(m(input), target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)

input:  tensor([[-2.4433,  0.8135,  0.2733, -0.7469, -0.0871],
        [-0.2786, -0.0351, -1.7871,  1.0199,  0.2018],
        [ 1.1733,  0.8662, -1.7969, -0.4139,  0.0683]], requires_grad=True)
target:  tensor([1, 0, 4])
output:  tensor(1.6014, grad_fn=<NllLossBackward0>)


In [53]:
target.shape

torch.Size([3])

In [55]:
input.shape

torch.Size([3, 5])

In [56]:
m(input).shape

torch.Size([3, 5])

In [54]:
yr_labels.shape

torch.Size([4606])

In [57]:
Xr_normalized.shape

torch.Size([4606, 71])

In [58]:
model(Xr_normalized).shape

torch.Size([4606, 3])

In [78]:
# real training loop
import copy
import tqdm
import numpy as np

# split
X_train, X_test, y_train, y_test = train_test_split(Xr_normalized, yr_onehot, train_size=0.7, shuffle=True)

n_epochs = 1500
batch_size = 450
batches_per_epoch = len(X_train) // batch_size

best_acc = - np.inf   # init to negative infinity
best_weights = None
train_loss_hist = []
train_acc_hist = []
test_loss_hist = []
test_acc_hist = []

for epoch in range(n_epochs):
    epoch_loss = []
    epoch_acc = []
    # set model in training mode and run through each batch
    model.train()
    with tqdm.trange(batches_per_epoch, unit="batch", mininterval=0) as bar:
        bar.set_description(f"Epoch {epoch}")
        for i in bar:
            # take a batch
            start = i * batch_size
            X_batch = X_train[start:start+batch_size]
            y_batch = y_train[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            y_batch_labeled = torch.argmax(y_batch, 1)
            loss = loss_fn(y_pred, y_batch_labeled)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
            # compute and store metrics
            acc = (torch.argmax(y_pred, 1) == torch.argmax(y_batch, 1)).float().mean()
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))
            bar.set_postfix(
                loss=float(loss),
                acc=float(acc)
            )
    # set model in evaluation mode and run through the test set
    model.eval()
    y_pred = model(X_test)
    ce = loss_fn(y_pred, torch.argmax(y_test, 1))
    acc = (torch.argmax(y_pred, 1) == torch.argmax(y_test, 1)).float().mean()
    ce = float(ce)
    acc = float(acc)
    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    test_loss_hist.append(ce)
    test_acc_hist.append(acc)
    if acc > best_acc:
        best_acc = acc
        best_weights = copy.deepcopy(model.state_dict())
    print(f"Epoch {epoch} validation: negative log likelihood loss={ce}, Accuracy={acc}")
    
model.load_state_dict(best_weights)

Epoch 0: 100%|██████████| 7/7 [00:00<00:00, 129.63batch/s, acc=0.262, loss=nan]


Epoch 0 validation: negative log likelihood loss=nan, Accuracy=0.2590448558330536


Epoch 1: 100%|██████████| 7/7 [00:00<00:00, 140.01batch/s, acc=0.262, loss=nan]


Epoch 1 validation: negative log likelihood loss=nan, Accuracy=0.2590448558330536


Epoch 2: 100%|██████████| 7/7 [00:00<00:00, 122.81batch/s, acc=0.262, loss=nan]


...

Epoch 1498: 100%|██████████| 7/7 [00:00<00:00, 92.11batch/s, acc=0.262, loss=nan]


Epoch 1498 validation: negative log likelihood loss=nan, Accuracy=0.2590448558330536


Epoch 1499: 100%|██████████| 7/7 [00:00<00:00, 92.10batch/s, acc=0.262, loss=nan]

Epoch 1499 validation: negative log likelihood loss=nan, Accuracy=0.2590448558330536


<All keys matched successfully>

In [ ]:
# plot the losses
import matplotlib.pyplot as plt

plt.plot(train_loss_hist, label="train")
plt.plot(test_loss_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("cross entropy")
plt.legend()
plt.show()

plt.plot(train_acc_hist, label="train")
plt.plot(test_acc_hist, label="test")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show()

In [ ]:
#OLD CODE

In [ ]:
#Gives accuracy and f1 score for using every stat possible
rfc=RandomForestClassifier()
Xr=ufc_fights[possible_stats]
yr=ufc_fights['method']

rfc.fit(Xr,yr)
accuracy = cross_val_score(rfc,Xr,yr,cv=3).mean()
precision = cross_val_score(rfc,Xr,yr,cv=3, scoring='precision_micro').mean()
recall = cross_val_score(rfc, Xr, yr, cv=3, scoring='recall_macro').mean()
print('Accuracy: '+str(accuracy),'F1 score: '+str(precision*recall/(precision+recall)))

In [ ]:
#scores a model
def model_score(dataframe,features, iloc_val = 3200):
    yyy=dataframe['method'].iloc[0:iloc_val]
    XXX=dataframe[features].iloc[0:iloc_val]
    XXXscaler = preprocessing.StandardScaler().fit(XXX)
    XXX_scaled = XXXscaler.transform(XXX) 
    rfc=RandomForestClassifier()
    return cross_val_score(rfc,XXX_scaled,yyy,cv=4).mean()
    
#CODE FOR THE GREEDY ALGORITHM FOR FEATURE SELECTION
def greedy(dataframe, features, subsetsize, iloc_val=3200):
    i=0
    best_stats=[]
    s=set(features)
    subsets=list(map(set, itertools.combinations(s, subsetsize))) #subsets of size (subsetsize)
    possible_stat_dict = {}
    scores={0:0}
    for stat_pair in subsets:
        possible_stat_dict[tuple(stat_pair)]=0
    while (i==0) or (scores[i]>scores[i-1]):
        i+=1
        for stat_pair in list(possible_stat_dict.keys()):
            stats_temp = best_stats+list(stat_pair)
            possible_stat_dict[tuple(stat_pair)]=model_score(ufc_fights,stats_temp,iloc_val)
        max_key = max(possible_stat_dict, key=possible_stat_dict.get)
        best_stats.extend(list(max_key))
        scores[i]=possible_stat_dict[max_key]
        possible_stat_dict.pop(max_key)
        print(best_stats,scores[i])
    return (best_stats[:-subsetsize], scores[i-1])

In [ ]:
#searches over all single stats for the highest scoring, then it trys to add one and sees if that scores higher
#this will keep adding stats as long as they improve the accuracy of the model
greedy(ufc_fights, possible_stats, 1)

In [ ]:
greedy(ufc_fights, possible_stats, 2)

In [ ]:
#random forest does way better for picking the method!
best_stats=['fighter_ko_losses_diff_2']
rfc=RandomForestClassifier()
Xr=ufc_fights[best_stats]
yr=ufc_fights['method']

rfc.fit(Xr,yr)
accuracy = cross_val_score(rfc,Xr,yr,cv=3).mean()
precision = cross_val_score(rfc,Xr,yr,cv=3, scoring='precision_micro').mean()
recall = cross_val_score(rfc, Xr, yr, cv=3, scoring='recall_macro').mean()
print('Accuracy: '+str(accuracy),'F1 score: '+str(precision*recall/(precision+recall)))

In [ ]:
#random forest does way better for picking the method!
best_stats=['fighter_ko_losses_diff_2', 'fighter_sub_losses_diff_2']
rfc=RandomForestClassifier()
Xr=ufc_fights[best_stats]
yr=ufc_fights['method']

rfc.fit(Xr,yr)
accuracy = cross_val_score(rfc,Xr,yr,cv=3).mean()
precision = cross_val_score(rfc,Xr,yr,cv=3, scoring='precision_micro').mean()
recall = cross_val_score(rfc, Xr, yr, cv=3, scoring='recall_macro').mean()
print('Accuracy: '+str(accuracy),'F1 score: '+str(precision*recall/(precision+recall)))

In [ ]:
#checking how the score changes with iloc value
for i in range(1,50):
    iloc_val = 100*i
    best_stats=['fighter_ko_losses_diff_2', 'fighter_sub_losses_diff_2']
    rfc=RandomForestClassifier()
    Xr=ufc_fights[best_stats].iloc[0:iloc_val]
    yr=ufc_fights['method'].iloc[0:iloc_val]

    rfc.fit(Xr,yr)
    accuracy = cross_val_score(rfc,Xr,yr,cv=3).mean()
    precision = cross_val_score(rfc,Xr,yr,cv=3, scoring='precision_micro').mean()
    recall = cross_val_score(rfc, Xr, yr, cv=3, scoring='recall_macro').mean()
    print(100*i, 'Accuracy: '+str(accuracy),'F1 score: '+str(precision*recall/(precision+recall)))

In [ ]:
#crazy... one single stat puts us at .51, higher than using every possible stat... but two gives no improvement at all

In [ ]:
num_subs = len([i for i in ufc_fights['method'] if i=='SUB'])
num_kos = len([i for i in ufc_fights['method'] if i=='KO/TKO'])
num_dec = len([i for i in ufc_fights['method'] if i=='DEC'])

In [ ]:
print('number of subs: '+str(num_subs))
print('number of kos: '+str(num_kos))
print('number of decisions: '+str(num_dec))

In [ ]:
num_fights=len(ufc_fights['method'])
print('percentage of subs: '+str(num_subs/num_fights))
print('percentage of kos: '+str(num_kos/num_fights))
print('percentage of decisions: '+str(num_dec/num_fights))

In [ ]:
# so picking DEC victory every single time would get us a score of .494. We are not really beating that by much...
# but we have a better f1 score than picking DEC every time. Having a higher f1 score is important since the 
# odds are better usually when you correctly pick KO or SUB